In [13]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

# Set up the WebDriver (e.g., Chrome)
# Make sure to provide the path to your WebDriver executable.
driver = webdriver.Chrome()

# Navigate to a website
url = 'https://example.com'  # Replace with your target website
driver.get(url)

seats = pd.read_excel('CED_2021_AUST.xlsx')
seats = seats[['CED_CODE_2021', 'CED_NAME_2021', 'STATE_NAME_2021']].drop_duplicates()
seats = seats.rename(columns={'CED_CODE_2021':'quickstats_code'})

seat_codes = seats['quickstats_code'].to_list()

ancestry_tables = []

for seat_code in seat_codes:
    try:
        url = 'https://www.abs.gov.au/census/find-census-data/quickstats/2021/CED' + seat_code
        driver.get(url)
        time.sleep(5)
        ancestry_table = pd.read_html(driver.find_element(By.XPATH, '/html/body/div[1]/main/div[3]/div/div[2]/div[3]/div[1]/div[4]/div[2]/div[1]/table').get_attribute('outerHTML'))
        ancestry_tables.append(
            {
                'seat_code': seat_code,
                'ancestry_table': ancestry_table
            }
        )
    except:
        pass

In [23]:
IN_voters = []
for seat in ancestry_tables:
    if 'Indian' in seat['ancestry_table'][0].iloc[:,0].to_list():
        index = seat['ancestry_table'][0].iloc[:,0].to_list().index('Indian')
        temp = {
            'Seat_code': seat['seat_code'],
            'IN_PROP': seat['ancestry_table'][0].iloc[index,2]
        }
        IN_voters.append(temp)

IN_voters = pd.DataFrame(IN_voters)

In [25]:
results = IN_voters.merge(seats, how='left', left_on='Seat_code', right_on='quickstats_code')
results.to_csv('IN_voters.csv')